# Problem Set 1
## Data generating process

We assume the general data generating process that fulfills the assumptions of the linear regression model: 

$$x_{i1}=1, x_{i2}:x_{iP} \sim \mathcal{N}(0,1)$$

$$y_{i}=\mathbf{x}_i'\boldsymbol{\beta}+\varepsilon_i$$

with $\boldsymbol{\beta}=\left(\begin{matrix} 1&0.5 &-0.5&-0.5&\ldots & -0.5&  \end{matrix}\right)$ and $\varepsilon_i \sim \mathcal{N}(0,1)$ for all $i$ and (initially) $n=1000$.

- Generate the outcome variable using P=2
- Estimate the parameter vector using OLS
- Calculate standard errors for the parameters

## Basic Simulation Study: Consistency 

With Consistency of $\hat{\boldsymbol{\beta}}$ we mean that 

$$\hat{\boldsymbol{\beta}}\overset{p}{\longrightarrow}\boldsymbol{\beta}$$
